In [2]:
print("Hello")

Hello


# install libraries

In [5]:
# pip install apify-client
# !pip install textblob

   ---------------------------------------- 0.0/624.3 kB ? eta -:--:--
   ---------------------------------------- 624.3/624.3 kB 5.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 8.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Extract some data

In [8]:
from apify_client import ApifyClient
import pandas as pd
from textblob import TextBlob  # For sentiment analysis

# Initialize Apify client with your API key
client = ApifyClient("")

# Correct Actor ID (replace with the latest one!)
ACTOR_ID = "maxcopell/twitter-scraper"  # Example (check Apify Store)
# Configure input
run_input = {
    "searchTerms": ["#vape OR #vaping OR 'disposable vape' lang:en"],
    "maxTweets": 500,
    "proxyConfig": {"useApifyProxy": True},
}

# Run the Actor
print("🚀 Starting Twitter scrape...")
try:
    run = client.actor(ACTOR_ID).call(run_input=run_input)
    
    # Fetch results
    tweets = []
    for item in client.dataset(run["defaultDatasetId"]).iterate_items():
        tweets.append({
            "text": item.get("text", ""),
            "likes": item.get("likes", 0),
            "retweets": item.get("retweets", 0),
            "username": item.get("user", {}).get("username", ""),
            "date": item.get("date", ""),
            "hashtags": item.get("hashtags", []),
        })

    # Save to CSV
    df = pd.DataFrame(tweets)
    df["sentiment"] = df["text"].apply(lambda x: TextBlob(x).sentiment.polarity)
    df.to_csv("vape_tweets.csv", index=False)
    print(f"✅ Saved {len(df)} tweets to 'vape_tweets.csv'!")

except Exception as e:
    print(f"❌ Error: {e}")

🚀 Starting Twitter scrape...
❌ Error: Actor with this name was not found
